In [ ]:
# Import necessary libraries
import geopandas as gpd
import json
import requests

# Initial Setup -----------------------------------------------------------

# Set StreetLight key and email
StlKey = "DgO7zCdYednFilSOR1dTX7jsmsdcoz31"
email = "chris.ryan@hdrinc.com"

# Set tmap_mode (if needed)
# tmap_mode("view")

proj = 6595

bufferSize = 5280 * 3
gridSize = 5280 * 0.33

# Read in shapefiles for stationPoints and stationArea
stationPoints = gpd.read_file("shape/stationPoints.shp")
stationPoints = stationPoints.to_crs(epsg=proj)
stationPoints = stationPoints.sort_values(by="name")

stationArea = gpd.read_file("shape/RailStationAreas.shp")
stationArea = stationArea.to_crs(epsg=proj)
stationArea = stationArea[["PropName"]].rename(columns={"PropName": "name"})
stationArea = gpd.sjoin(stationPoints, stationArea, op="within")[["name", "type"]]
stationArea = stationArea.sort_values(by="name")

# Map check
basemaps = ["CartoDB.Positron", "OpenStreetMap.Mapnik", "Esri.WorldImagery"]
stationArea.plot(color="orange", alpha=0.5)
stationPoints.plot(column="type", legend=True, cmap="Set1")

# NOTE: Confirm that both GeoDataFrames have the same number of locations and are in the same order for 'name'
(stationPoints["name"] == stationArea["name"]).all()

# Create StL Zone Sets ----------------------------------------------------

# Create Station Area Zones
stationArea_ZS = stationArea[["name"]].copy()
stationArea_ZS["id"] = range(1, len(stationArea_ZS) + 1)
stationArea_ZS["is_pass"] = 0
stationArea_ZS["direction"] = None
stationArea_ZS = stationArea_ZS.to_crs(epsg=4326)
stationArea_ZS = stationArea_ZS.set_z(0)

# Convert to JSON
zone_set_name = "StationAreas2021-09-17"
zones = json.loads(stationArea_ZS.to_json())

body = {
    "insight_login_email": email,
    "geom_type": "polygon",
    "zone_set_name": zone_set_name,
    "zones": zones
}

# Convert to final JSON file
bodyJSON = json.dumps(body)

# API call to create Zone Set
def api_call(url, data):
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    return response

url = f"https://insight.streetlightdata.com/api/v2/zone_sets?key={StlKey}"
api_call(url, bodyJSON)

# Loop to create hex grid zone set for each station area
allGrid = gpd.GeoDataFrame(crs=proj)
for _, station in stationPoints.iterrows():
    buffer = station.geometry.buffer(bufferSize)
    grid = buffer.sindex.query(buffer, predicate="intersects")
    grid = buffer[grid].difference(stationArea.loc[_]["geometry"])
    grid = grid.rename(columns={"geometry": "name"}).reset_index(drop=True)
    grid["id"] = range(1, len(grid) + 1)
    grid["is_pass"] = 0
    grid["direction"] = None
    grid = grid.set_z(0)

    allGrid = allGrid.append(grid)

    # Convert to JSON
    zone_set_name = station["name"] + str(grid["id"].iloc[0])
    zones = json.loads(grid.to_json())

    body = {
        "insight_login_email": email,
        "geom_type": "polygon",
        "zone_set_name": zone_set_name,
        "zones": zones
    }

    # Convert to final JSON file
    bodyJSON = json.dumps(body)

    # API call to create Zone Set
    url = f"https://insight.streetlightdata.com/api/v2/zone_sets?key={StlKey}"
    api_call(url, bodyJSON)

allGrid.plot(alpha=0.5)

# Save data for Step 2.0
allGrid.to_file("shape/stationArea.geojson", driver="GeoJSON")
